# **3214. Year on Year Growth Rate**

**Table: user_transactions**
``` sql
+------------------+----------+
| Column Name      | Type     | 
+------------------+----------+
| transaction_id   | integer  |
| product_id       | integer  |
| spend            | decimal  |
| transaction_date | datetime |
+------------------+----------+
```
The transaction_id column uniquely identifies each row in this table.

Each row of this table contains the transaction ID, product ID, the spend amount, and the transaction date.

Write a solution to calculate the year-on-year growth rate for the total spend for each product.

The result table should include the following columns:

year: The year of the transaction.

product_id: The ID of the product.

curr_year_spend: The total spend for the current year.

prev_year_spend: The total spend for the previous year.

yoy_rate: The year-on-year growth rate percentage, rounded to 2 decimal places.

Return the result table ordered by product_id,year in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**user_transactions table:**
``` sql
+----------------+------------+---------+---------------------+
| transaction_id | product_id | spend   | transaction_date    |
+----------------+------------+---------+---------------------+
| 1341           | 123424     | 1500.60 | 2019-12-31 12:00:00 |
| 1423           | 123424     | 1000.20 | 2020-12-31 12:00:00 |
| 1623           | 123424     | 1246.44 | 2021-12-31 12:00:00 |
| 1322           | 123424     | 2145.32 | 2022-12-31 12:00:00 |
+----------------+------------+---------+---------------------+
```
**Output:**
``` sql
+------+------------+----------------+----------------+----------+
| year | product_id | curr_year_spend| prev_year_spend| yoy_rate |
+------+------------+----------------+----------------+----------+
| 2019 | 123424     | 1500.60        | NULL           | NULL     |
| 2020 | 123424     | 1000.20        | 1500.60        | -33.35   |
| 2021 | 123424     | 1246.44        | 1000.20        | 24.62    |
| 2022 | 123424     | 2145.32        | 1246.44        | 72.12    |
+------+------------+----------------+----------------+----------+
```
**Explanation:**

**For product ID 123424:**
- **In 2019:**
  - Current year's spend is 1500.60
  - No previous year's spend recorded
  - YoY growth rate: NULL
**In 2020:**
  - Current year's spend is 1000.20
  - Previous year's spend is 1500.60
  - YoY growth rate: ((1000.20 - 1500.60) / 1500.60) * 100 = -33.35%
**In 2021:**
  - Current year's spend is 1246.44
  - Previous year's spend is 1000.20
  - YoY growth rate: ((1246.44 - 1000.20) / 1000.20) * 100 = 24.62%
**In 2022:**
  - Current year's spend is 2145.32
  - Previous year's spend is 1246.44
  - YoY growth rate: ((2145.32 - 1246.44) / 1246.44) * 100 = 72.12%

Note: Output table is ordered by product_id and year in ascending order.

**Solution-1:**
``` sql
select
year(transaction_date) as year, product_id, sum(spend) as curr_year_spend,
lag(sum(spend),1) over(PARTITION BY product_id order by year(transaction_date)) as prev_year_spend,
round((sum(spend) - lag(sum(spend),1) over(PARTITION BY product_id order by year(transaction_date))) *100 /lag(sum(spend),1) over(PARTITION BY product_id order by year(transaction_date)),2) as yoy_rate
from user_transactions 
group by 1,2
ORDER BY product_id, year

**Solution-2:**
``` sql
WITH CUR_YEAR_SPEND AS (
    SELECT 
        YEAR(transaction_date) AS year,
        product_id,
        SUM(spend) AS curr_year_spend
    FROM user_transactions
    GROUP BY year, product_id
),
CUR_AND_PREV_YEAR_SPEND AS (
    SELECT 
        year,
        product_id,
        curr_year_spend,
        LAG(curr_year_spend, 1) OVER(PARTITION BY product_id ORDER BY year ASC) AS prev_year_spend
    FROM CUR_YEAR_SPEND
)
SELECT
    year,
    product_id,
    curr_year_spend,
    prev_year_spend,
    ROUND(100 * (curr_year_spend - prev_year_spend) / prev_year_spend, 2) AS yoy_rate
FROM CUR_AND_PREV_YEAR_SPEND
ORDER BY product_id, year

**Solution-3:**
``` sql

WITH info AS (
    SELECT product_id, SUM(spend) AS spend, YEAR(transaction_date) AS year 
    FROM user_transactions 
    GROUP BY product_id, YEAR(transaction_date)
    )

SELECT a.year, a.product_id, a.spend AS curr_year_spend, b.spend AS prev_year_spend, 
ROUND((a.spend - b.spend) * 100.0 / b.spend, 2) AS yoy_rate 
FROM info a 
LEFT JOIN info b 
ON a.product_id = b.product_id AND a.year = b.year + 1 ORDER BY product_id, year